<a href="https://colab.research.google.com/github/thePhukan/Xception_EEG-/blob/parent/Xception_feature_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
print(tf.__version__)

from tensorflow import keras
tf.random.set_seed(42)

import numpy as np
np.random.seed(42)

import matplotlib.pyplot as plt
%matplotlib inline

import glob
import PIL
from PIL import Image
import pandas as pd
from skimage.io import imread, imshow
import PIL
from PIL import Image
from sklearn import preprocessing
import os

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
2.12.0


# The original dataset "**emotions.csv**" can be found in this [link](https://www.kaggle.com/datasets/birdy654/eeg-brainwave-dataset-feeling-emotions)

In [10]:
dataset = pd.read_csv('Path to the origianl dataset: emotions.csv')
X = dataset.iloc[:, 0:2548].values
y = dataset.iloc[:,2548].values

In [ ]:
imgFiles = sorted(glob.glob("Path to the directory where you have saved the generated spectrograms/*.jpg"), key=os.path.getmtime)
for items in imgFiles:
  print(items)

In [14]:
X = []

for fName in imgFiles:

  # Prepare the dataset and populate X and y
  X_i =Image.open(fName) # tiny_FR/sunflower/1715303025_e7065327e2.jpg (500, 333)
  X_i = X_i.resize((299,299)) # To make them approriate to Xception model when using Transfer Learning
  X_i = np.array(X_i) / 255.0 # Normalize to range 0.0 to 1.0 (not stretching, only scaling)
  # X_i = tf.image.per_image_standardization(X_i)

  X.append(X_i)

In [ ]:
from sklearn.preprocessing import LabelEncoder
lEncoder = LabelEncoder()
y = lEncoder.fit_transform(y)

print(y)
print(lEncoder.classes_)

In [ ]:
X = np.array(X)
y = np.array(y)

print(X.shape)
print(y.shape)

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    shuffle=False)


In [18]:
X_train_std = X_train
X_test_std = X_test

## Transfer Learning using Xception

In [19]:
# Retrieve the Xception model from Keras

Xception_model = keras.applications.xception.Xception()

91884032/91884032 [==============================] - 0s 0us/step


In [ ]:
Xception_model.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [20]:
# Take only the feature extractor part

base_model = keras.applications.xception.Xception(weights='imagenet',
                                                  include_top=False, pooling='max')

83683744/83683744 [==============================] - 1s 0us/step


In [ ]:
base_model.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_3[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [ ]:
X_train_features = []
from itertools import chain
for i in X_train_std:
  i = np.expand_dims(i, axis=0)
  temp = base_model.predict(i)
  temp2 = temp.flatten()
  X_train_features.append(temp2)


# We use this to save csv file

In [22]:
import csv
with open('Path to directory where you want to save the extracted features/Xception_Features.csv', 'w') as f:
	# using csv.writer method from CSV package
	write = csv.writer(f)
	write.writerows(X_train_features)
